In [1]:
# DASH PLOTLY LIBRARIES- FOR UI
import dash
import dash_table
import datetime
import dash_core_components as dcc
import dash_html_components as html

from sklearn.linear_model import LinearRegression
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.graph_objs as go
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import math

#FOR TIME SERIES ANALYSIS
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

df = pd.read_csv('Gemini_BTCUSD_daily.csv')
dft = pd.read_csv('Gemini_ETHUSD_daily.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([

    html.Div([

        # Title
        html.Div([
          html.H1(children='Crypto Currency Prediction')
        ], style={'textAlign': 'center'}),

        # Drop down and data table
        html.Div([
            html.H3(children='Crypto Currency Type'),

            dcc.Dropdown(
                id='table-dropdown',
                options=[
                    {'label': 'Bitcoin', 'value': 'BTC'},
                    {'label': 'Ethirium', 'value': 'ETH'}
                ],
                value='BTC'
            ),

            dcc.Markdown('''
            #### Choose one of the crypto currency
            
             Data of crypto currency daily price including it's highest, lowest, open, close, etc.
            All data is taken from [Crypto Data](http://www.cryptodatadownload.com/data/northamerican/).
            
            '''),

            html.H3(children='Data Table'),

            dash_table.DataTable(
                id='table',
                data=df.to_dict('records'),
                columns=[{'id': c, 'name': c} for c in df.columns],
                fixed_rows={'headers': True, 'data': 0},
                style_cell={'width': '150px'}
            )],
        ),

        html.Div([
            html.H3(children='Data Graph'),

            dcc.Graph(id='data-graph')
        ]),

        html.Div([
            html.H3(children='Candlestick'),

            dcc.Graph(id='data-candlestick')
        ]),

        html.Div([
            html.H2(children='Prediction using linear regression'),

            dcc.Markdown(''' 

            '''),

            dcc.Graph(id='graph')
        ]),

        html.Div([
            html.H2(children='Time series prediction'),

            dcc.Markdown('''
            #### Time series prediction with deep learning


            '''),

            dcc.Graph(id='graph-learn')
        ]),

    ]),
])


@app.callback(Output('table', 'data'), [Input('table-dropdown', 'value')])
def update_rows(value):
    # dff = df[df['some-column'] == value]
    if value == 'BTC':
        return df.to_dict('records')
    elif value == 'ETH':
        return dft.to_dict('records')


@app.callback(Output('data-graph', 'figure'), [Input('table-dropdown', 'value')])
def update_data_graph(value_dropdown):

    if value_dropdown == 'BTC':
        df_graph = df
        text = 'Bitcoin'
    elif value_dropdown == 'ETH':
        df_graph = dft
        text = 'Ethirium'

    return {
        'data': [go.Scatter(
            mode='lines',
            x=df_graph['Date'],
            y=df_graph['Open'],
            text=text,
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'blue'}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': 'Date',
                'tickvals': [df_graph["Date"].iloc[0], df_graph["Date"].iloc[250], df_graph["Date"].iloc[500],
                             df_graph["Date"].iloc[750], df_graph["Date"].iloc[1000], df_graph["Date"].iloc[-1]],
                # 'tickformat': '%m/%y'
            },
            yaxis={
                'title': 'Price : USD',
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }


@app.callback(Output('data-candlestick', 'figure'), [Input('table-dropdown', 'value')])
def update_graph(value_dropdown):
    if value_dropdown == 'BTC':
        df_to_candlestick = df
    elif value_dropdown == 'ETH':
        df_to_candlestick = dft
    return {
        'data': [go.Candlestick(
            x=df_to_candlestick['Date'],
            open=df_to_candlestick['Open'],
            high=df_to_candlestick['High'],
            low=df_to_candlestick['Low'],
            close=df_to_candlestick['Close'],
        )],
    }


def datetime_to_float(d):
    epoch = datetime.datetime.utcfromtimestamp(0)
    total_seconds =  (d - epoch).total_seconds()
    # total_seconds will be in decimals (millisecond precision)
    return total_seconds


##-----------------LINEAR REGRESSION MODEL---------------

@app.callback(Output('graph', 'figure'), [Input('table-dropdown', 'value')])
def update_graph(value_dropdown):
    if value_dropdown == 'BTC':
        df_to_graph = df
        df_graph = df
        text = 'Bitcoin'
        X = df.iloc[:, 0].values.reshape(-1, 1)  # values converts it into a numpy array
        Y = df.iloc[:, 4].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
    elif value_dropdown == 'ETH':
        df_to_graph = dft
        df_graph = dft
        text = 'Ethirium'
        X = dft.iloc[:, 0].values.reshape(-1, 1)  # values converts it into a numpy array
        Y = dft.iloc[:, 4].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column

    # OBJECT CREATION 
    
    linear_regressor = LinearRegression()  # create object for the class
    linear_regressor.fit(X, Y)  # perform linear regression
    Y_pred = linear_regressor.predict(X)  # make predictions

    df_for_graph = pd.DataFrame(Y_pred, columns=['Prediction'])

    return {
        'data': [go.Scatter(
            name='Linear Regression Prediction',
            mode='lines',
            x=df_to_graph['Date'],
            y=df_for_graph['Prediction'],
            text='Linear Regression Line',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'blue'}
            }
        ),
            go.Scatter(
                name='Actual Data',
                mode='lines',
                x=df_graph['Date'],
                y=df_graph['Open'],
                text=text,
                marker={
                    'size': 15,
                    'opacity': 0.5,
                    'line': {'width': 0.5, 'color': 'red'}
                })
        ],
        'layout': go.Layout(
            xaxis={
                'title': 'Date',
                'tickvals': [df_to_graph['Date'].iloc[0], df_to_graph["Date"].iloc[250], df_to_graph["Date"].iloc[500],
                             df_to_graph["Date"].iloc[750], df_to_graph["Date"].iloc[1000], df_to_graph["Date"].iloc[-1]
                             ],
            },
            yaxis={
                'title': 'Price : USD',
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }


# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)


#----------TIME SERIES ANALYSIS-------------------------------------

@app.callback(Output('graph-learn', 'figure'), [Input('table-dropdown', 'value')])
def update_graph_learn(value_dropdown):
    if value_dropdown == 'BTC':
        csv_name = 'Gemini_BTCUSD_daily.csv'
        df_to_learn = df
    elif value_dropdown == 'ETH':
        csv_name = 'Gemini_ETHUSD_daily.csv'
        df_to_learn = dft

    fd = pd.read_csv('Gemini_BTCUSD_daily.csv')

#print(fd['Close'])

# fd['Close'].plot()
    data = fd.iloc[:, 5:6]
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    data_scaler = scaler.fit_transform(data)
# training_set = data_scaler[:600]
# test_set = data_scaler[601:]

    X_train = []
    Y_train = []

    for i in range(50, 600):
        X_train.append(data_scaler[i-50:i, 0])
        Y_train.append(data_scaler[i])

    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
# X_train = training_set[0: len(training_set)-1]
# Y_train = training_set[1: len(training_set)]
#

    X_test = []
    Y_test = []
    for i in range(600, len(data_scaler)):
        X_test.append(data_scaler[i-50:i, 0])
        Y_test.append(data_scaler[i])

    X_test = np.array(X_test)
    Y_test = np.array(Y_test)

# X_test = test_set[0: len(test_set)-1]
# Y_test = test_set[1: len(test_set)]


#
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    X_train = np.reshape(X_train, (len(X_train), 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (len(X_test), 1, X_test.shape[1]))

    from keras.models import Sequential
    from keras.layers import Activation, Dense
    from keras.layers import LSTM
    from keras.layers import Dropout

    model = Sequential()
    model.add(LSTM(20, return_sequences=True,  input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dropout(.1))
    model.add(LSTM(40))
# model.add(Dropout(.2))
# model.add(LSTM(60, return_sequences=True))
# model.add(Dropout(.2))
# model.add(LSTM(50))
    model.add(Dense(units=1))


    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['binary_accuracy', 'categorical_accuracy'])
    model.summary()
    history = model.fit(X_train, Y_train, epochs=200, batch_size=24, verbose=2, shuffle="batch", validation_data=(X_test, Y_test))

    LSTM_prediction = model.predict(X_test)
    predicted_price = scaler.inverse_transform(LSTM_prediction)
    actual_price = scaler.inverse_transform(Y_test)

    return {
        'data': [go.Scatter(
            name='Actual Data',
            mode='lines',
            x=df_to_learn['Date'],
            y=data_scaler['Data'],
            text='Actual Data',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'blue'}
            }
        ),
            go.Scatter(
                name='Training',
                mode='lines',
                x=df_to_learn['Date'],
                y=trainPredictPlot['Train'],
                text='Training',
                marker={
                    'size': 15,
                    'opacity': 0.5,
                    'line': {'width': 0.5, 'color': 'green'}
                }),

            go.Scatter(
                name='Test',
                mode='lines',
                x=df_to_learn['Date'],
                y=testPredictPlot['Test'],
                text='Test',
                marker={
                    'size': 15,
                    'opacity': 0.5,
                    'line': {'width': 0.5, 'color': 'red'}
                })
        ],
        'layout': go.Layout(
            xaxis={
                'title': 'Date',
                'tickvals': [df_to_learn['Date'].iloc[0], df_to_learn["Date"].iloc[250], df_to_learn["Date"].iloc[500],
                             df_to_learn["Date"].iloc[750], df_to_learn["Date"].iloc[1000], df_to_learn["Date"].iloc[-1]
                             ],
            },
            yaxis={
                'title': 'Price : USD',
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }


if __name__ == '__main__':
    app.run_server(debug=True)
    



Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

c:\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
